In [48]:
# Import libraries
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import requests
import json
import datetime
import sqlalchemy
import random
import numpy as np
from datetime import datetime, date, timedelta
from pytz import timezone


def lambda_handler(event, context):
    
       def get_city_info(cities):
        
        #Create the dataframe
        city_info = pd.DataFrame(columns=["city_id", "city_name", "country", "population"])
        
        #Create a for loop to extract the data from wikipedia
        for city_url in cities:
            r = requests.get(city_url)
            soup = bs(r.content, 'html.parser')
            city_name = soup.find("span", class_="mw-page-title-main").string
            country = soup.find("td", class_="infobox-data").string
            pop = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
            
            #create a dataframe from the data extracted
            city_to_append = pd.DataFrame({
                "city_name": [city_name],
                "country": [country],
                "population": [pop]
            })
            
            #Concatenate the dataframe city_info with the city_to_append dataframe
            city_info = pd.concat([city_info, city_to_append], ignore_index=True)
            
            #Create a unique id for each city
            city_info['city_id'] = city_info['city_name'].map({'Berlin': 1, 'Manchester': 2, 'Barcelona': 3})
            
            #Update the data type of the city_id to numeric
            city_info['city_id'] = pd.to_numeric(city_info['city_id'])
        
        return city_info
    
    #Set required cities   
    cities = [
        "https://en.wikipedia.org/wiki/Berlin",
        "https://en.wikipedia.org/wiki/Manchester",
        "https://en.wikipedia.org/wiki/Barcelona"
    ]
    
    #Assign the function to the city_info dataframe
    city_info = get_city_info(cities)

    #Create population and city tables
    population = city_info[["city_id", "population"]]
    city = city_info[["city_id", "city_name", "country"]]
    
    
    def get_weather(cities):
        # Set API key
        api_key = "56a8f02f74cf9d8370f9d4e0a58aa754" 
        
        # Create the dataframe
        weather_data = pd.DataFrame(columns=["city_id", "city_name", "date", "temperature", "weather"])
    
        # Create a for loop to pull weather data for each city from the API
        for city in cities:
            url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
            print(f"Fetching data for {city}")
            weather = requests.get(url)
    
            # Read and set as JSON
            weather_json = weather.json()
    
            # Create a for loop to extract the data for each time slot in the forecast
            for i in range(len(weather_json['list'])):
                city_name = city
                date = weather_json['list'][i]['dt_txt']
                temp = weather_json['list'][i]['main']['temp']
                weather_description = weather_json['list'][i]['weather'][0]['main']
            
                weather_to_append = pd.DataFrame({
                    "city_name": [city_name],
                    "date": [date],
                    "temperature": [temp],
                    "weather": [weather_description]
                })
            
                weather_data = pd.concat([weather_data, weather_to_append], ignore_index=True)
    
            # Create a unique id for each city
            weather_data['city_id'] = weather_data['city_name'].map({'Berlin': 1, 'Manchester': 2, 'Barcelona': 3})
        
        # Drop the city_name column   
        weather_data = weather_data.drop(columns=['city_name'])
    
        # Update the data format
        weather_data["date"] = pd.to_datetime(weather_data['date'])
        weather_data['city_id'] = pd.to_numeric(weather_data['city_id'])
        
        return weather_data
    
    # Set required cities
    cities = ["Barcelona", "Manchester", "Berlin"]

    # Assign the function to the city_info dataframe
    weather_data = get_weather(cities)
    
    def get_airports():
    # Create a dictionary with airports, airport codes, and the unique ID for each
        airports_data = {
            'city_id': [1, 2, 3],
            'airport': [
                'Berlin Brandenburg Airport',
                'Manchester Airport',
                'Josep Tarradellas Barcelona Airport'
            ],
            'airport_icao': ['EDDB', 'EGCC', 'LEBL']
        }
        
        global airports

        # Assign the dictionary to an airports DataFrame
        airports = pd.DataFrame(airports_data)
    
        return airports

# Call the function to create the airports DataFrame
    airports = get_airports()
    
#Flights
    #create function
    def get_flights(icao_list):

        today = datetime.now().date()
        tomorrow = (today + timedelta(days=1))
        
        flights_data = []
        flights_json = {} 

        #create for loop for each ICAO
        for icao in icao_list:
            times = [["00:00", "11:59"], ["12:00", "23:59"]]
           
            for time in times:
                url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
                querystring = {"withLeg": "true", "direction": "Arrival", "withCancelled": "false", "withCodeshared": "true", "withCargo": "false", "withPrivate": "false"}
                flight_headers = {
                    "X-RapidAPI-Key": "0b7cffd425mshbf932b1b5f7e633p187a96jsna5aedb4ce276",
                    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }
                response = requests.request("GET", url, headers=flight_headers, params=querystring)
                print(response)
                flights_json = response.json()
                
                for flight in flights_json['arrivals']:
                    flights_dict = {}
                    flights_dict['arrival_icao'] = icao
                    flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
                    flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
                    flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
                    flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
                    flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
                    flights_dict['airline'] = flight['airline'].get('name', None)
                    flights_dict['flight_number'] = flight.get('number', None)
                    flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
                    flights_data.append(flights_dict)
        
        return pd.DataFrame(flights_data)

    icaos = ['EDDB','EGCC','LEBL']
    get_flights(icaos)
    flights_data = get_flights(icaos)
    
    # Create connection to MySQL
    schema = "gans"
    host = "wbs-project3-db.ctnijtvo60ph.eu-west-2.rds.amazonaws.com"
    user = "admin"
    password = "LexiHarley0323"
    port = 3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    # Create city table in MySQL
    city.to_sql('city',
                if_exists='append',
                con=con,
                index=False)
    # Create the weather_data table in MySQL
    weather_data.to_sql('weather',
                if_exists='append',
                con=con,
                index=False)
                
    airports.to_sql('airports', 
              if_exists='append', 
              con=con, 
              index=False)
    #create the population table in MySQL
    population.to_sql('population', 
              if_exists='append', 
              con=con, 
              index=False)
    #create the flights_data table in MySQL
    flights_data.to_sql('flights', 
              if_exists='append', 
              con=con, 
              index=False)

    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

Fetching data for Barcelona
Fetching data for Manchester


C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9728\1226569671.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_data = weather_data.append({
C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9728\1226569671.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_data = weather_data.append({
C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9728\1226569671.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_data = weather_data.append({
C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9728\1226569671.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_data = weather_data.append({
C:\Users\Jod

Fetching data for Berlin


C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9728\1226569671.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_data = weather_data.append({
C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9728\1226569671.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_data = weather_data.append({
C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9728\1226569671.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_data = weather_data.append({
C:\Users\JodieHorrocks\AppData\Local\Temp\ipykernel_9728\1226569671.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  weather_data = weather_data.append({
C:\Users\Jod

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


1680